## Intro

---------------------------------

![GAN](./image/gan1.jpg)

Generative model 은 랜덤한 `Latent code`를 줬을 때, 이미지를 생성함.  


어떤 training의 데이터가 있을 때 그 분포를 학습하는 것이 목적이다.

*실제 이미지에 대한 분포를 학습시킬 수 있을까?*

실제 이미지에 대한 확률분포함수(확률밀도함수)가 밑의 이미지의 형태로 존재한다고 가정한다.

![그래프](./image/gan2.jpg)

사람 얼굴 데이터로 예를 들어 그래프를 설명한다.  
안경을 낀 남자 이미지에 해당하는 픽셀을 대표하는 값을 x1 벡터라고 한다. 그 이미지가 적게 등장한다면 그 x1에 해당하는 확률밀도값은 작게 된다.
흑발 여자 이미지가 그보다 많이 등장한다면, 그에 해당하는 벡터 x2는 더 높고, 마찬가지로 금발여자가 많이 등장한다면 x3은 높게 된다.
기계가 생성한 이상한 얼굴 이미지 x4는 학습 데이터에 존재하지 않아 매우 낮다. 

![그래프2](./image/gan3.jpg)

generative모델이 하고자 하는 것은 실제 데이터의 분포를 잘 근사하는 모델을 만드는 것이다.
파란색이 실제 고차원 픽셀을 의미하는 데이터 분포고, 빨간색은 모델이 생성한 것이다.

***모델의 목표 : 두 개의 확률분포간 차이를 줄여주는 것***



##Generative Adversarial Networks 

----------------------------------------------------------

![대체 텍스트](./image/gan4.jpg)

*G 모델을 학습하는 것이 최종목표이고, 그 전에 D모델을 학습한다.*

- D모델은 진짜 이미지를 진짜로 가짜 이미지를 가짜로 구별하는 것이 목표.  
input : 이미지 벡터  
outuput : binary (진짜 1/가짜 0. sigmoid를 통해 0.5 기준으로 classification)

- G모델은 D의 output이 1이 되도록 학습하므로 학습을 시킬수록 진짜같은 가짜 이미지를 생성해낸다.  
input : 랜덤한 코드  
output : 이미지

![수식](./image/gan5.jpg)

- discriminator 관점.
  - 좌측 항 (진짜 이미지를 받았을 때) :  
  gan의 loss함수(objective 함수) 를 최대화하는 것이 목표  
x-p data(x) : 확률 밀도함수. 실제 학습 데이터가 있으면 x값을 하나씩 뽑아옴.  
D는 진짜 x를 받았을 때 1에 가까운 값을 내놓아야 한다.(D(x)는 0~1사이 값.)  
 log D(x)를 최대화시켜야 함.  가장 최대 값은 D(x)가 1일 때 0으로 최대가 됨. D(x)가 0일 때는 -무한대로 최소가 됨.
 
  - 오른쪽 항 (가짜 이미지를 받았을 때) :  
 밑의 그림에서 랜덤벡터 z가 generator의 입력 값으로 들어가고 G가 가짜 이미지를 생성한다.  
 z는 가우시안 분포나 100차원짜리 벡터로 샘플링한 랜덤한 벡터.  
G가 생성한 이미지를 D가 받았을 때는 0에 가까운 값을 내놔야 한다.  
 log(1-x)수식은 x값이 0일 때 최대값이 되기 때문에, 이 수식이 최대화되도록 학습시킨다.  
 
- Generator관점
 
 우측의 항을 최소화 시켜야 함.  
 진짜 이미지를 D가 학습할 때는 G가 쓰이지 않으므로, 왼쪽 항에는 관여할 수 없음.   
 우측의 항은 D(G(Z)) = 1일 때, 최소가 됨.  
 D가 가짜 이미지를 받았을 때, 1에 가까운 값을 내놓도록 학습을 시킴.





In [0]:
#실제로 동작하는 코드는 아님!

import torch
import torch.nn. as nn

#input : 784 (MNIST이미지 28*28을 편 것)
#hidden : 128 (가정한 것)
#output : 1 (0 아니면 1이니까)
D = nn.Sequential(
	nn.Linear(784 ,128),
	nn.ReLU(),
	nn.Linear(128, 1),
	nn.Sigmoid())
	
#input : 100 (latent가 100차원이라고 가정)  
#hidden : 128이라고 가정
#output : 784(이미지랑 같아야 하니까)
G = nn.Sequential(
	nn.Linear(100, 128),
	nn.ReLU(),
	nn.Linear(128, 784),
	nn.Tanh()) # 생성된 값이 -1 ~ 1
	 
# Binary Cross Entropy Loss(h(x), y), Sigmoid Cross Entropy Loss 함수라고도 불림. 
# -ylogh(x)-(1-y)log(1-h(x))
# h(x) : prediction.값. y : label값 (1또는 0)
# y = 1 인 경우는 오른쪽 항이 사라짐.
# 실제 간 학습 때는 h(x)대신 D(x)를 넣어줄 것인데, y에 1을 넣고 h(x)에 D(x)를 넣어주면 
# -logD(x)만 남음. -를 지우고 maximize하나 -내버려두고 minimize하나 마찬가지임. 앞에서 D수식의 왼쪽 항과 동일하게 됨.
# y값이 0이 되면 왼쪽 항이 지워지고, 오른쪽만 남음 -log(1-h(x)). 이는 아까 봤던 오른쪽 항과 동일
criterion = nn.BCELoss()

d_optimizer = torch.optim.Adam(D.parameters(), lr=0.01)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.01)
# 충돌하기에 2개의 optimizer를 설정

#x는 784의 차원, z는 100차원 랜덤벡터라고 가정
# x : (batch_size, 782) z : (batch_size, 100)

#보통 discriminator학습 하고, generator학습 하고 반복시킴.

while True:
	# train D
	loss = criterion(D(x), 1) + criterion(D(G(z)), 0) #왼쪽 : 진짜 이미지, 오른쪽 : 가짜 이미지
	loss.backward() # 모든 weight에 대해 gradient값을 계산
	d_optimizer.step() #앞에서 정의한 adam옵티마이저로 gradient 학습.
	
	# train G
  #실제로 generator학습을 할 때는 Discriminator는 고정시켜줘야 함.
	loss = criterion(D(G(z)), 1) 
	loss.backward()
	g_optimizer.step() # generator의 파라미터를 학습 (discriminator는 학습 x)
   

>tanh 함수를 사용하는 이유?

>실제 이미지를 학습할 때는 -1~1사이로 normalization을 해준 후 학습한다.   
위와 동일하게, 진짜 이미지의 범위를 맞추기 위해 tanh를 쓰는데, 
사실 generator가 학습하는 과정에서 -1보다 작은 값, 1보다 큰 값을 내보내면 안된다는 것을 학습함.  
그러므로 딱히 꼭 써야하는 것은 아님.



![대체 텍스트](./image/gan6.jpg)

처음에 generator가 생성할 때, 굉장히 형편없는 이미지를 만듦. 그 때, D(G(z))는 거의 0에 수렴함.  
그 때의 기울기를 보면, 기울기의 절대값이 꽤 낮음(=학습 속도가 느림) 그래서 수식을 바꿔줘서 기울기를 높게 해줌  
= 초반에 discriminator가 가짜라고 확신할 수 있는 상황을 빠르게 벗어날 수 있도록 함.

이것이 잘 돌아가는 이유  
*최적화하는 것이 서로 다른 확률분포간의 차이를 줄여주기 때문에 실제 Generator가 실제와 가까운 이미지를 만들 수 있다!*




## Variants of GAN

### DCGAN 

- CNN을 사용해서 discriminator를 생성하고 deconvolutional network를 통해서 generator를 만든 모델  
 descriminator에서 convolution을 쓰고, generator에서 deconvolution(transpose convolution) 사용.  
- 핵심 :  pooling layer를 사용하지 않음.  
unpooling시에 blocking한 이미지를 생성하기 때문.  
대신 Stride가 2 이상인 convolution과 deconvolution을 사용함
- Adam Optimizer의 모멘텀 텀이 0.5, 0.999 2개가 있음.  
64x64를 생성할 때 저 파라미터를 사용하면 성능이 좋아서 이렇게 사용중. 거즘 고정
- Generator에 들어가는 latent vector를 통해 연산을 할 수 있음(Word2vec같이!)  
Z vector가 선형적 관계를 가짐  
예를 들어, 안경낀 남자에서 안경없는 남자를 빼면, 안경만 남음. 그 상태에서 여자 사진을 더하면 안경낀 여자가 나오게 됨.  
이렇게 z벡터가 선형적인 관계를 가짐. z벡터를 어떻게 구할지는 나중에 설명함.

### LSGAN (Least Squares GAN )

![대체 텍스트](./image/gan10.jpg)

- 기존 gan loss에서는 discriminator를 속이기만 하면 됨.
- 파란색 선은 discriminator의 decision boundary.  
 0.5를 기준으로 그보다 높으면 진짜, 낮으면 가짜. 그 기준선을 나타냄.  
기준선 아랫쪽이 진짜라고 예측을 함.  
빨간색이 진짜 데이터고 파란색이 가짜 데이터임
- 빨간색에 가까운 파란색은 generator가 잘 만든 것.  
- 핑크색은 discriminator를 완벽하게 속인 것. 
- 핑크색이 좋은 것은 아닌게, 빨간 색인 진짜 데이터와 떨어져 있기 때문. discriminator를 속이기는 했는데 좋지는 못한 데이터.  
여기서 gradient vanishing문제가 생김. = 완벽하게 discriminator를 속였어도, 그게 실제 데이터와 비슷하다고 보장할 수 없음.
- discriminator에서 Sigmoid를 없애고, 기존의  cross entropy loss를 least square loss(L2 loss)로 바꿈.
이렇게 바꿈으로써 1보다 작은 값이 나오던, 큰 값이 나오던 generator부분에서 deicision boundary가 1에 가깝도록 만듬.  
세 번재 그림(LSGAN)을 보면, 핑크색을 위로 끌어서 올린 것을 볼 수 있다.  
이미지의 퀄리티가 좋아진다.  

### SGAN(Semi-Supervised GAN)

![대체 텍스트](./image/gan9.jpg)

- discriminator가 진짜, 가짜 + 클래스를 구분함.  
예를 들어, MNIST를 보면 클래스 개수가 10개인데, 거기에 fake 클래스를 추가함.  
위의 그림은 진짜 5 이미지를 받았을 때, 5라고 출력하는 것  
밑의 이미지는 latent vector와 label이 담긴 one-hot vector를 input으로 넣어주면, 그에 해당하는 가짜 이미지를 생성함.  
discriminator부분에서는 이를 fake class로 판별.
- 이름이 붙은 이유는 위의 경우에는 supervised고, 아래의 경우에는 unsupervised이기 때문.
- 클래스에 따른 이미지를 얻을 수 있기 때문에 기존 gan에서 조금 더 업그레이드 됨. 물론 label된 train 데이터 셋이 필요함.  
z벡터는 그대로 두고, one-hot vector만 수정하면 이미지가 변함.
- z는 처음에는 무의미했다가, 학습이 많이 지나고 나서 z를 보면 선형성을 가지게 됨. 의미있는 벡터가 됨.  
ex ) z가 조금 변하면 캐릭터가 조금 변하고, z가 많이 변하면 캐릭터의 변화가 큼.


### ACGAN(Auxiliary Classifier GAN)

- discriminator가 해야 하는 일이 두 가지임.  
진짜인지 가짜인지 .  
어떤 클래스인지.  
따라서 중간층인 convolution layer는 multi-task임.
- 마지막단인 FC layer는 하나는 sigmoid, 하나는 softmax로 독립적으로 학습.

- 가짜 이미지를 가지고 학습을 하는 경우에는 latent벡터를 넣어주고, one-hot vector 형태로 클래스 정보를 넣어줌.  
학습을 하면 할수록 generator가 좋은 이미지를 생성하기 때문에, discriminator가 classification까지 하는 경우 generator가 해주는 역할이 data agumentation의 역할을 할 수 있음.
- discriminator에 초점을 맞춰, generator가 좀 안좋은 이미지를 만든 것은 noise라고 생각하고, discriminator가 잘 classification할 수 있도록 해줌.




## Extensions of GAN

### CycleGAN

![대체 텍스트](./image/gan11.jpg)

- 얼룩말<->말, 여름<->겨울. 이런 식으로 이미지를 변할 수 있음.

- 그림에서는 얼룩말 그림을 말로 바꾸고자 함. 갑자기 포즈가 바뀌거나 하는 것은 원하지 않음. 이것이 테스크의 목표임.

- discriminator의 경우, 얼룩말 이미지를 말로 바꾸고 싶기 때문에 말의 이미지를 주고 이를 참이라고 함.

- generator의 경우에는 latent vector를 받지 않고, 어떤 이미지를 받게 됨. encoder decoder같이.  
얼룩말 이미지를 받으면, discriminator의 결과가 참으로 나오도록 해야하기 때문에 이미지를 말로 변화시킴.

- 포즈가 바뀌는 것을 막기 위해 얼룩말 이미지를 말로 바꾼 후, 말 이미지를 다시 얼룩말로 바꿈.   
서로 다른 네트워크이기 때문에, 만약 얼룩말 이미지가 밑처럼 뛰고 있는 말 이미지로 바뀌면, 다시 얼룩말로 바꾸기 힘들 것이다.  
다시 복원하기 위해서는 모양이 최대한 덜 바뀌어야 한다. 이 점을 뉴럴 네트워크가 학습할 때 이용함.

- data가 최소한 만 개 정도는 있어야 잘 돌아갈 것.

### StackGAN

![대체 텍스트](./image/gan12.jpg)

- 어떠한 텍스트를 주고, 그 텍스트에 해당하는 이미지를 생성하는 것.  
ex) 새가 난다. -> 새가 나는 이미지 생성

- generator가 랜덤한 벡터를 받아서 이미지를 생성하고, 그 이미지가 discriminator에 들어가서 가짜 진짜 분간. (G2이전의 그림)  
그런데 랜덤한 벡터를 받아서 한번에 이미지를 생성하는 것이, 64x64사이즈는 잘 되는데 128x128 등은 잘 안됨.

- stackGAN의 아이디어는 64x64를 먼저 생성하고, 그것을 받아서 업샘플링 하게끔 또다른 generator를 두는 것  
z벡터로부터 바로 고해상도의 사진을 만드는 것보다 훨씬 나음.  
최종 이미지는 너무 크거나 작으면 잘 동작하지 않고, 128~256이 젤 좋음.

> 64x64 이미지가 하얀색 새가 들어가면,  128x128도 꼭 하얀색 새가 나와야 할까?  
G2입장에서는 하얀색 새가 들어와도 다른 색을  ouput으로 해서 discriminator를 속일 수도 있지 않을까?

>그렇게 되지는 않음. 따로 제한을 두지 않았음에도 하얀색 새를 넣으면 하얀색만 g2의 output으로 나옴.  
D2를 속이기 위해 색을 바꾸는 것보다는, 크기 등을 바꾸는 것이 더 쉬워서 그런 것이 아닐까 추측함.




### Visual Attribute Transfer

- input으로 두 이미지를 줌.  
ex) input으로 모나리자와 아바타 사진을 주면, 모나리자같이 생긴 아바타와 아바타같이 생긴 모나리자를 output으로 줌.

### User-Interactive Image Colorization

- 사용자가 원하는 색의 input을 넣어주면 색이 칠해짐.  
ex) 새는 빨간색으로 찍고, 나무는 초록색으로 찍으면 색이 알아서 칠해짐. 퀄높은 색칠공부처럼

## Future of GAN

(발표자의 주관이 많이 들어감)

### BEGAN (Boundary Equilibrium GAN)

- 지금까지는 loss가 없어서 사용자가 이미지를 보고 잘 되는지 아닌지 봐야 했음.
- gan에도 그에 맞는 loss함수를 만들어서 관찰했더니 해당 로스함수가 줄어들 때 학습이 잘되더라  
(하지만 이것은 휴리스틱하게 나온 결과)
- 모델의 하이퍼파라미터에 따라서도 값들이 변함.
- 한계점 : 복잡한 구조에서만 사용할 수 있다

### Reconstruction Loss

- 지금까지는 간에서 batch normalization을 사용했는데, weight normalization을 사용하면 좋다는 것.

- z는 처음에 랜덤하게 샘플링 한 뒤, 이미지를 생성한다.  
L2 loss를 계산해서, 그것을 backpropagation을 해서 z를 찾음.  
처음에 제대로 된 이미지를 생성할 수 있는 z벡터를 찾는다면 모델이 일반화를 해서 잘 생성을 했다 는 것을 보여줄 수 있음.

- 단점 : z를 학습시켜야 하는 것 자체가 단점임. 굉장히 휴리스틱하고 느림.

### Deconvolution Checkboard Artifacts

- deconvolution할 때, 오른쪽으로 두 칸씩 띄게 되는데 그럼 겹치는 부분이 생김. = output을 만들 때, 불균형하게 만들게 됨.  

- 단점은 체크보드 패턴으로, 이상한 이미지가 생성되게 됨. (왜 굳이 두 칸씩 띄는지 모르겠는데, 대부분 논문이 2칸씩만 띈다고 함)

- 그러므로 deconvolution이 아니라 resize-convolution을 사용하자!

- deconvolution은 upsampling하는 것 자체를 학습하지만, resize는 rule-based로 upsampling을 함.

- 그 뒤에 convolution의 stride =1로 하고 필터링을 여러 겹을 쌓음.
장점은 체크보드가 생기지 않고, 골고루 적용하게 됨

- BEGAN에서는 resize-convolution을 사용함. 다른 네트워크에는 deconvolution이 더 잘된다고 함

### Machine Translation

- 간을 기존의 supervised learning에 적용시킬 수 있지 않을까

- seq2seq모델은 영어를 줫을 때 한글로 번역함.

- 번역할 때, 뜻은 같지만, 여러가지의 말로 번역할 수 있음.  
supervised learning은 정해진 정답인 하나의 말로만 번역하지만, 간을 적용해서 여러가지의 말로 나오게 할 수 있지 않을까 
ex) buy -> 사다, 구입하다, 구매하다 등..

- descriminator의 경우에는 영어하고 한글 문장을 받아서, 그 두개가 같은 뜻을 받으면 1이라고 한다.  
generator가 가짜 한글을 만들면, 그것을 가짜라고 함.

- generator는 영어를 받았을 때, 같은 뜻의 다른 말인 한글을 생성함.
